# Laboratorio de regresión - 5

Santiago Reyes Castillo  
17 de febrero, 2025  
745826   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [363]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy import random

In [257]:
data = pd.read_excel('Motor Trend Car Road Tests.xlsx')
data = data.drop(columns = ["model"])

In [258]:
train = data.copy()
lista = []

In [259]:
from sklearn.preprocessing import StandardScaler

In [260]:
for i in range(len(y)):
    
    x = pd.DataFrame(data.drop(columns=["mpg"]))
    y = pd.DataFrame(data["mpg"])

    x_train = x.drop(index=i)
    y_train = y.drop(index=i)
    
    x_test = x.iloc[[i],:]
    y_test = y.iloc[[i],:]
    
    scaler = StandardScaler()
    scaler.fit(x_train)

    X_std_train = scaler.transform(x_train)
    X_test_std = scaler.transform(x_test)

    model = LinearRegression()

    model.fit(X_std_train, y_train)

    y_pred = model.predict(X_test_std)

    
    mse = mean_squared_error(y_test, y_pred)

    lista.append(mse)
  
   


In [261]:
mses = np.array(lista)

In [262]:
mean = mses.mean()
mean

12.181558006901941

In [263]:
std = mses.std()
std

17.067399871888586

Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [287]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [323]:
X = pd.DataFrame(X)
X.columns=[housing.feature_names]

y = pd.DataFrame(y)
y.columns=[housing.target_names]

In [338]:
california = X.copy()
california["MedHouseVal"] = y
a = california.index

In [371]:
rand_index = np.random.choice(a, round(len(california)), replace=False)

In [373]:
california_random = california.iloc[rand_index, :]
california_random.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
10365,4.0221,16.0,5.594796,1.078067,1649.0,3.065056,33.61,-117.69,2.362
4747,2.2725,39.0,3.421053,1.024561,723.0,2.536842,34.05,-118.34,1.406
9106,2.1413,38.0,4.679245,1.037736,261.0,4.924528,34.62,-118.02,0.969
2243,3.2143,25.0,4.989293,0.989293,1215.0,2.601713,36.82,-119.79,0.930
11549,6.0609,25.0,6.582792,1.025974,3613.0,2.932630,33.74,-118.01,2.649


In [448]:
rebanada = len(y)/10
rebanada = int(rebanada)
rebanada

2064

In [493]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
10365,4.0221,16.0,5.594796,1.078067,1649.0,3.065056,33.61,-117.69
4747,2.2725,39.0,3.421053,1.024561,723.0,2.536842,34.05,-118.34
9106,2.1413,38.0,4.679245,1.037736,261.0,4.924528,34.62,-118.02
2243,3.2143,25.0,4.989293,0.989293,1215.0,2.601713,36.82,-119.79
11549,6.0609,25.0,6.582792,1.025974,3613.0,2.932630,33.74,-118.01
...,...,...,...,...,...,...,...,...
10281,4.0714,21.0,6.414583,1.060417,1431.0,2.981250,33.89,-117.82
8023,5.0719,37.0,5.684874,0.970588,1331.0,2.796218,33.84,-118.12
337,2.2500,46.0,4.320225,1.028090,693.0,3.893258,37.74,-122.17
18226,2.8203,8.0,3.325843,0.930337,856.0,1.923596,37.41,-122.09


In [479]:
X = california_random.drop(columns=["MedHouseVal"])
y = california_random["MedHouseVal"]

mses = []

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

for i in range(10) :
    y_test = y.iloc[rebanada*i : rebanada*(i+1), :]
    y_train = y.drop(index=y_test.index)
    
    X_test = X.iloc[rebanada*i : rebanada*(i+1), :]
    X_train = X.drop(index=X_test.index)

    scaler = StandardScaler()
    scaler.fit(X_train)

    X_std_train = scaler.transform(X_train)
    X_test_std = scaler.transform(X_test)

    model = LinearRegression()

    model.fit(X_std_train, y_train)

    y_pred = model.predict(X_test_std)
    
    mse = mean_squared_error(y_test, y_pred)

    mses.append(mse)
    


C:\Users\reyes\AppData\Local\Temp\ipykernel_25272\3736458520.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = california_random.drop(columns=["MedHouseVal"])


In [489]:
mses = np.array(mses)
mses.mean()

0.527859803776329

In [491]:
mses.std()

0.03358618294612784

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3